In [96]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
# hack the paths, no package install needed
import sys
import os
repo_dir = '/media/user/repos/bulk-euclid-cutouts'
os.path.isdir(repo_dir)
sys.path.insert(0,repo_dir)

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
import os

import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
import pandas as pd

from bulk_euclid.external_targets import pipeline

### Set Pipeline Options and Load External Targets

In [ ]:
# you can use this config and external target list for a minimal test on six targets within one tile...

cfg_dict = dict(
    # TODO change this path to wherever you want to save everything
    base_dir='/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version',
    name='external_targets_debug',

    sas_environment='IDR', # for Q1, REG for Regression, or OTF for On-The-Fly
    release_name=None, # for no specific release or e.g. 'F-006' for one release 
    release_priority=['CALBLOCK_PV-005_R2', 'CALBLOCK_PV-005_R3', 'F-003_240321', 'F-003_240612' , 'F-006', 'Q1_R1'],  # use the highest-index release first, if available
    bands=['VIS', 'NIR_Y'], #  , 'NIR_J'
    auxillary_products=['MERPSF','MERRMS'],

    overwrite_fits=False,
    download_method='datalabs_path',  # to use datalabs data (Q1 only) or 'sas' for download
    fits_outputs=True,
    jpg_outputs= ['sw_mtf_vis_only', 'sw_mtf_vis_y_j', 'sw_arcsinh_vis_only'],  # three example processing options, see cutout_utils.py for more options
     
    
    # these will be filled in automatically using on base_dir
    download_dir=None,
    tile_dir=None,
    cutout_dir=None,
    fits_dir=None,
    jpg_dir=None,
    sanity_dir=None
    
)
cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc
pipeline.create_folders(cfg)  # fill in those missing paths

external_targets = pd.DataFrame([
    {
      'id_str': 'a',
      'target_ra': 92.8371844681179,
      'target_dec': -47.8868920683699,
      'target_field_of_view': 20},
     {
      'id_str': 'b',
      'target_ra': 92.836691187603,
      'target_dec': -48.1117628801209,
      'target_field_of_view': 20},
     {
      'id_str': 'd',
      'target_ra': 92.835560566775,
      'target_dec': -47.978437467137,
      'target_field_of_view': 20},
     {
      'id_str': 'e',
      'target_ra': 92.8351734327103,
      'target_dec': -48.1054532354171,
      'target_field_of_view': 20},
     {
      'id_str': 'f',
      'target_ra': 92.8343976295701,
      'target_dec': -47.9993017111107,
      'target_field_of_view': 20}
    
])

INFO:root:Saving to /media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version/external_targets_debug


In [109]:
len(cfg.bands)

2

In [101]:
len(external_targets)

5

In [102]:
assert not any(external_targets['id_str'].duplicated())

In [103]:
external_targets

,id_str,target_ra,target_dec,target_field_of_view
0,a,92.837184,-47.886892,20
1,b,92.836691,-48.111763,20
2,d,92.835561,-47.978437,20
3,e,92.835173,-48.105453,20
4,f,92.834398,-47.999302,20


### Authenticate

In [ ]:
# type in your COSMOS credentials
from astroquery.esa.euclid.core import EuclidClass
Euclid = EuclidClass(environment=cfg.sas_environment)
Euclid.login()

INFO:astroquery:Login to Euclid TAP server


INFO: Login to Euclid TAP server [astroquery.esa.euclid.core]


User:  mwalms01
Password:  ········


INFO:astroquery:OK
INFO:astroquery:Login to Euclid data service


INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid data service [astroquery.esa.euclid.core]


INFO:astroquery:OK
INFO:astroquery:Login to Euclid cutout service


INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid cutout service [astroquery.esa.euclid.core]


INFO:astroquery:OK


INFO: OK [astroquery.utils.tap.core]


### For each target, find a tile that covers it

In [104]:
targets_with_tiles = pipeline.get_matching_tiles(cfg, external_targets)
targets_with_tiles

INFO:astroquery:Query finished.
INFO:root:Found 2514 query results
INFO:root:Begin target/tile cross-match
INFO:root:Matched 5 targets to 1 tiles
INFO:root:Targets with tile matches: 5


INFO: Query finished. [astroquery.utils.tap.core]


,id_str,target_ra,target_dec,target_field_of_view,tile_index,release_name
0,a,92.837184,-47.886892,20,102021539,F-006
1,b,92.836691,-48.111763,20,102021539,F-006
2,d,92.835561,-47.978437,20,102021539,F-006
3,e,92.835173,-48.105453,20,102021539,F-006
4,f,92.834398,-47.999302,20,102021539,F-006


In [105]:
targets_with_tiles['release_name'].value_counts()

release_name
F-006    5
Name: count, dtype: int64

In [106]:
targets_with_tiles = targets_with_tiles.sort_values('tile_index')[:1000]

The speed of the pipeline depends on the number of tiles to download. It takes about 30 seconds per tile, per band, per product. If you request two bands and four products (as above), each tile takes about 4 minutes.

In [107]:
targets_with_tiles['tile_index'].nunique()

1

### For every tile, download the flux and auxillary (PSF, RMS, BKG) data and make cutouts around each target in that tile

In [116]:
pipeline.make_cutouts(cfg, targets_with_tiles)

INFO:root:Tile 102021539, 0 of 1
INFO:astroquery:Query finished.
INFO:root:Found 2 query results


INFO: Query finished. [astroquery.utils.tap.core]


INFO:root:Downloaded: {'NIR_Y': {'FLUX': '/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version/external_targets_debug/tiles/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE102021539-677258_20240405T223525.219021Z_00.00.fits', 'MERRMS': '/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version/external_targets_debug/tiles/EUC_MER_MOSAIC-NIR-Y-RMS_TILE102021539-F9D4BF_20240405T222015.990175Z_00.00.fits', 'MERPSF': '/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version/external_targets_debug/tiles/EUC_MER_CATALOG-PSF-NIR-Y_TILE102021539-989050_20240406T034820.708751Z_00.00.fits'}, 'VIS': {'FLUX': '/media/home/my_workspace/repos/bulk-euclid-cutouts/tests/test_external_targets/test_notebook_version/external_targets_debug/tiles/EUC_MER_BGSUB-MOSAIC-VIS_TILE102021539-F91316_20240405T233444.877933Z_00.00.fits', 'MERPSF': '/media/home/my_workspace/repos/bulk-euclid-cutouts/t